In [4]:
import os
import pandas as pd
from autoemulate.core.compare import AutoEmulate
from autoemulate.core.sensitivity_analysis import SensitivityAnalysis
from ModularCirc import BatchRunner
import torch
figsize = (9, 5)

In [5]:
import torch

# Check if MPS is available
if torch.backends.mps.is_available():
    device = "mps"
    print("Using MPS (Metal Performance Shaders) for GPU acceleration")
elif torch.cuda.is_available():
    device = "cuda"
    print("Using CUDA for GPU acceleration")
else:
    device = "cpu"
    print("Using CPU")


Using MPS (Metal Performance Shaders) for GPU acceleration


In [ ]:
param_filename = 'parameters_naghavi_constrained_fixed_T_v_tot_v_ref_lower_k_pas'

n_samples = 256

simulation_out_path = f'../outputs/simulations/output_{n_samples}_samples_{param_filename}/'

parameters_json_file = os.path.join(simulation_out_path, 'parameters.json')

# Load the simulation input parameters
input_params = pd.read_csv(os.path.join(simulation_out_path, f'input_samples_{n_samples}.csv'))

# Load the summary statistics
summary_stats = pd.read_csv(os.path.join(simulation_out_path, f'simulations_summary.csv'))

output_to_emulate = 'p_ao_max'
# create a folder called emulators inside simulation outpath
emulators_path = os.path.join(simulation_out_path, 'emulators', output_to_emulate)
os.makedirs(emulators_path, exist_ok=True)


FileNotFoundError: [Errno 2] No such file or directory: '../outputs/simulations/output_2048_samples_parameters_naghavi_constrained_fixed_T_v_tot_v_ref_lower_k_pas/input_samples_2048.csv'

In [ ]:
# Use ModularCirc's BatchRunner to condense parameter names. This requires setting up the sampler first.
br = BatchRunner()

br.setup_sampler(parameters_json_file)

br._parameters_2_sample

{'ao.r': (195.0, 325.0),
 'ao.c': (0.22499999999999998, 0.375),
 'art.r': (843.75, 1406.25),
 'art.c': (2.25, 3.75),
 'ven.r': (6.75, 11.25),
 'ven.c': (84.975, 141.625),
 'av.r': (4.5, 7.5),
 'mv.r': (3.0749999999999997, 5.125),
 'la.E_pas': (0.22499999999999998, 0.375),
 'la.E_act': (0.41250000000000003, 0.6875),
 'la.k_pas': (0.00666, 0.03),
 'lv.E_pas': (0.75, 1.25),
 'lv.E_act': (2.25, 3.75),
 'lv.k_pas': (0.00666, 0.03)}

In [ ]:
# Get column names for parameters up to 'lv.k_pas'
parameter_names = list(input_params.columns[:input_params.columns.get_loc('lv.k_pas') + 1])

input_params[parameter_names]

,ao.r,ao.c,art.r,art.c,ven.r,ven.c,av.r,mv.r,la.E_pas,la.E_act,la.k_pas,lv.E_pas,lv.E_act,lv.k_pas
0,305.576111,0.364705,1047.778645,2.796825,7.379755,116.738799,5.767604,3.872886,0.253016,0.650535,0.027086,1.247329,3.465806,0.025985
1,224.490389,0.267449,1391.318553,3.568193,9.100033,91.786620,6.305379,4.954608,0.372932,0.446568,0.008090,0.806839,2.924290,0.006675
2,254.249478,0.327790,906.121586,2.608205,10.275423,104.035536,5.166425,4.290832,0.334445,0.542466,0.017685,0.985828,3.109324,0.020852
3,271.680109,0.230533,1251.458715,3.003151,8.699872,129.094902,6.925526,3.321711,0.286747,0.613231,0.020454,1.052777,2.523997,0.013266
4,285.041178,0.313907,1283.209757,3.239476,7.965886,122.133654,4.714354,4.508952,0.315288,0.459171,0.014723,1.100807,3.203127,0.011534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,284.548763,0.280885,937.668583,2.274293,11.184050,100.716070,6.303583,4.347080,0.363766,0.499886,0.017793,1.043770,3.045726,0.025803
252,271.220424,0.338484,1044.139211,2.510426,10.546397,93.751423,5.312585,3.483537,0.243544,0.619790,0.023176,1.122927,2.318208,0.022429
253,254.773662,0.285758,1403.746764,3.104593,8.746949,125.452918,6.602922,4.565447,0.344561,0.415554,0.014911,0.932217,3.327014,0.018254
254,224.981812,0.300398,904.267854,2.695387,7.816004,141.183684,4.706651,4.672173,0.306291,0.521659,0.010833,0.861206,2.756544,0.027508


In [ ]:
Y = summary_stats[output_to_emulate].values

In [ ]:
# Turn x into a pytorch tensor
x = torch.tensor(input_params[parameter_names].values, dtype=torch.float32)

# Do the same with Y
y = torch.tensor(Y, dtype=torch.float32)

In [ ]:
print(x.shape, y.shape)

torch.Size([256, 14]) torch.Size([256])


In [ ]:
ae = AutoEmulate(x = x,
                 y = y,
                 models=["GP", "MLP"],
                 device=device)

Comparing models:   0%|          | 0.00/1.00 [00:00<?, ?model/s]

Comparing models: 100%|██████████| 1.00/1.00 [00:14<00:00, 14.1s/model]


In [11]:
best = ae.best_result()
print(best.model_name)

GaussianProcess


In [12]:
best.params

{'mean_module_fn': <function autoemulate.emulators.gaussian_process.mean.constant_mean(n_features: int | None, n_outputs: torch.Size | None) -> gpytorch.means.constant_mean.ConstantMean>,
 'covar_module_fn': <function autoemulate.emulators.gaussian_process.kernel.rbf_plus_linear(n_features: int | None, n_outputs: torch.Size | None) -> gpytorch.kernels.kernel.Kernel>,
 'epochs': 100,
 'lr': 0.5,
 'likelihood_cls': gpytorch.likelihoods.multitask_gaussian_likelihood.MultitaskGaussianLikelihood,
 'scheduler_cls': torch.optim.lr_scheduler.LRScheduler,
 'scheduler_kwargs': {'policy': 'ReduceLROnPlateau',
  'patience': 5,
  'factor': 0.5}}

In [ ]:
ae.save(model_obj=best,
        path=emulators_path,
        use_timestamp=True)

PosixPath('../outputs/simulations/output_256_samples_parameters_naghavi_constrained_fixed_T_v_tot_v_ref_lower_k_pas/GaussianProcess_0_20250820_110540')

In [14]:
ae.summarise()


,model_name,x_transforms,y_transforms,params,rmse_test,r2_test,r2_test_std,r2_train,r2_train_std
0,GaussianProcess,[StandardizeTransform()],[StandardizeTransform()],{'mean_module_fn': <function constant_mean at ...,1.749858,0.998393,0.00041,0.999949,0.000008
